In [25]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

import models as ms

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.metrics import mean_squared_error

from sklearn import set_config
from joblib import dump, load

In [26]:
df_train_raw = pd.read_csv('/home/blue/general-assembly/dsir-824/submissions/projects/project-2-master/datasets/train.csv')

df = df_train_raw.copy()

In [27]:
# This will use the entire dataset
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [28]:
params = {
    'lasso__alpha': np.arange(20,30),
    'lasso__fit_intercept': [False],
    'lasso__max_iter': [-1]
},


In [29]:
# Instantiate class
ms = ms.ModelSelection(X, y)

In [36]:
pipe = make_pipeline(ms.preprocessing, Lasso())

In [37]:
gs = GridSearchCV(pipe, params, n_jobs=-1)

In [38]:
scores, objects = ms.evaluate_grid_search([gs])

In [39]:
scores

[0.9090195083380985]

In [ ]:
objects[0].best_params_

In [ ]:
#dump(objects[0], '/home/blue/general-assembly/dsir-824/submissions/projects/project-2-master/overfit-')

In [ ]:
ms.calculate_rsme(preprocessing, objects)

In [ ]:
# switch to class function

In [ ]:
predictions = objects[0].predict(df_test)

In [ ]:
predictions = pd.DataFrame(predictions)
predictions = predictions.rename({0:'SalePrice'}, axis=1)
predictions = predictions.join(df_test['Id'])
predictions = predictions[['Id', 'SalePrice']]

In [ ]:
#predictions.to_csv('/home/blue/general-assembly/dsir-824/submissions/projects/project-2-master/predictions/920-lasso-alpha-20-predictions.csv', index=False)

In [ ]:
predictions